# **Библиотеки**

# все для людей, которые меняют окружение 

In [ ]:
# !pip install matplotlib numpy scipy sympy ortools quantecon 

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 8.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 8.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 6.0 MB/s eta 0:00:0000:0100:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 6.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 7.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 8.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 8.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16/16 [ortools]5/16 [ortools]ib]


In [9]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Polygon
from ortools.linear_solver import pywraplp
from quantecon.optimize.linprog_simplex import linprog_simplex
from scipy.optimize import linprog
from sympy.solvers.simplex import lpmax
from sympy import Eq, Symbol

# **Задание_1**

Рассмотрим задачу, дуальную к задаче инвестиционного фонда


$$
\begin{aligned}
\min_{p_1, \dots, p_7} \;& 100000p_1 - 20000p_4 - 20000p_5 - 20000p_6 + 50000p_7 \\
\text{при условиях:} \quad 
& p_1 + p_2 + p_3 \ge 3.9, \\
& p_2 + p_7 \ge 1.3, \\
& p_1 - 1.06p_2 + p_4 = 0, \\
& p_2 - 1.06p_3 + p_5 = 0, \\
& p_4 + p_6 = 1.06, \\
& p_{1,2,3} \; \text{— свободные}, \quad 
p_{4,5,6} \le 0, \quad 
p_7 \ge 0.
\end{aligned}
$$

Задача не соответствует виду, который требуется для функции linprog_simplex.
   Преобразуйте ее соответствующим образом и решите при помощи метода linprog_simplex.

### Коротко — что делает каждый массив

### Общая идея
Код формулирует и решает задачу линейного программирования методом симплекса, затем восстанавливает физические переменные и печатает результаты.

### b_eq
- **b_eq = np.array([0, 0, 1.06])** — правая часть системы равенств A_eq @ x = b_eq; три значения задают требуемые суммы/балансы для трёх равенств

### A_eq
- **A_eq** — матрица коэффициентов для равенств; каждая строка соответствует одному уравнению вида сумма(коэффициент_i * x_i) = соответствующее значение из b_eq
- Структура строк задаёт связи между переменными p1,p1_,p2,p2_,p3,p3_,p4,p5,p6,p7 в трёх балансных уравнениях.

### b_ub
- **b_ub = np.array([-3.9, -1.3, 0])** — правая часть неравенств вида A_ub @ x ≤ b_ub; три числа задают верхние пределы для трёх ограничений

### A_ub
- **A_ub** — матрица коэффициентов для неравенств; каждая строка задаёт линейное неравенство между переменными (A_ub @ x ≤ b_ub)
- Первая строка связывает первые шесть переменных в одно неравенство, вторая и третья строки включают переменную p7 (десятая переменная)

### c_d
- **c_d** — вектор коэффициентов целевой функции c^T x, которую linprog минимизирует; значения задают вклад каждой переменной в целевую функцию.
- Отрицательные и положительные знаки влияют на стремление алгоритма уменьшить/увеличить соответствующие переменные; если задача реально максимизация, нужно было подать -c_d

### Вызов решателя
- **res = linprog_simplex(c_d, A_ub=..., b_ub=..., A_eq=..., b_eq=...)** — запускает оптимизацию: минимизация c_d^T x при ограничениях A_ub x ≤ b_ub и A_eq x = b_eq и x ≥ 0.
- **res** содержит решение: **res.x** (вектор переменных), **res.fun** (значение целевой функции), флаги успешности и сообщения

### Распаковка и восстановление переменных
- `p1, p1_, p2, p2_, p3, p3_, p4, p5, p6, p7 = res.x` — присваивает найденные неотрицательные компоненты.
- `P1 = p1 - p1_`, `P2 = p2 - p2_`, `P3 = p3 - p3_` — восстанавливает исходные переменные, допускающие знак, из их положительной и отрицательной частей (x = x_plus - x_minus).

### Печать результатов
- `print(f"p1 = {-P1:.4f}, ...")` и последующие строки — форматированный вывод восстановленных P1, P2, P3 и значений p4..p7 с инвертированным знаком 
- `print(f"Максимум -z = {res.fun:.4f}")` — выводит значение минимизированной функции res.fun

In [11]:
b_eq = np.array([0, 0, 1.06])

A_eq = np.array([
    [-1.00,  1.00,  1.06, -1.06,  0.00,  0.00, -1.00,  0.00,  0.00,  0.00],
    [ 0.00,  0.00, -1.00,  1.00,  1.06, -1.06,  0.00, -1.00,  0.00,  0.00],
    [ 0.00,  0.00,  0.00,  0.00, -1.00,  1.00,  0.00,  0.00, -1.00,  0.00],
])

b_ub = np.array([-3.9, -1.3, 0])

A_ub = np.array([
    [1, -1,  1, -1,  1, -1, 0, 0, 0, 0],
    [0,  0,  1, -1,  0,  0, 0, 0, 0, 1],
    [0,  0,  0,  0,  0,  0, 0, 0, 0, 1],
])

c_d = np.array([100_000, -100_000, 0, 0, 0, 0, -20_000, -20_000, -20_000, 50_000])

res = linprog_simplex(c_d, A_ub=A_ub, b_ub=b_ub,A_eq=A_eq,b_eq=b_eq)
res

p1, p1_, p2, p2_, p3, p3_, p4, p5, p6, p7 = res.x
P1 = p1 - p1_
P2 = p2 - p2_
P3 = p3 - p3_

print(f"p1 = {-P1:.4f}, p2 = {-P2:.4f}, p3 = {-P3:.4f}")
print(f"p4 = {-p4:.4f}, p5 = {-p5:.4f}, p6 = {-p6:.4f}, p7 = {-p7:.4f}")
print(f"Максимум -z = {res.fun:.4f}")


p1 = 1.3780, p2 = 1.3000, p3 = 1.2264
p4 = -0.0000, p5 = -0.0000, p6 = -0.1664, p7 = -0.0000
Максимум -z = -141128.3019
